In [1]:
from example import read_games
recs = list(read_games())

In [2]:
from collections import defaultdict
topkeys = defaultdict(int)
for rec in recs:
    for k in rec['data'].keys():
        topkeys[k] += 1
print(list(sorted(topkeys.keys())))

['about_the_game', 'achievements', 'background', 'categories', 'controller_support', 'demos', 'detailed_description', 'developers', 'dlc', 'drm_notice', 'ext_user_account_notice', 'genres', 'header_image', 'is_free', 'legal_notice', 'linux_requirements', 'mac_requirements', 'metacritic', 'movies', 'name', 'package_groups', 'packages', 'pc_requirements', 'platforms', 'price_overview', 'publishers', 'recommendations', 'release_date', 'required_age', 'reviews', 'screenshots', 'short_description', 'steam_appid', 'support_info', 'supported_languages', 'type', 'website']


In [3]:
for ky,tot in sorted(topkeys.items()):
    count = 0
    uniq = set()
    mn, mx = None, None
    
    for rec in recs:
        val = rec['data'].get(ky, None)
        if not val:
            continue
        count += 1
        if isinstance(val, dict):
            val = "<DICT>"
        elif isinstance(val, list):
            val = "<LIST>"
        else:
            val = str(val).strip()
        uniq.add(val)
        if mn is None or val < mn:
            mn = val
        if mx is None or val > mx:
            mx = val
    print("%-25s %8d %8d %20s %20s" % (ky, tot, len(uniq), str(mn)[:20], str(mx)[:20]))
        

about_the_game               12037    11233 !nsane is a nod to t 日本語バージョンダウンロード開始されまし
achievements                  6615        1               <DICT>               <DICT>
background                   12037    11187 http://cdn.akamai.st http://cdn.akamai.st
categories                   11813        1               <LIST>               <LIST>
controller_support            3391        1                 full                 full
demos                         1134        1               <LIST>               <LIST>
detailed_description         12037    11234 !nsane is a nod to t 日本語バージョンダウンロード開始されまし
developers                   11402        1               <LIST>               <LIST>
dlc                           2076        1               <LIST>               <LIST>
drm_notice                      81       42 Act Control<br>5 mac           simple key
ext_user_account_notice        134       83 1C Game Studios Acco       www.doak.co.za
genres                       11294        1           

In [24]:
from random import choice
from collections import Counter
from pprint import PrettyPrinter
pp = PrettyPrinter(width=120)
pprint = pp.pprint

def fields(name):
    for rec in recs:
        v = rec['data'].get(name, None)
        if v:
            yield v
            
field_values = list(fields('support_info'))
def out(v):
    pprint(v)
    print("")
out(field_values[0])
out(field_values[-1])
for _ in range(10):
    out(choice(field_values))


# def scan_values():
#     for fv in field_values:
#         free, purchase, subscript = False, False, False
#         for pg in fv:
#             if pg.get('is_recurring_subscription', '') == 'true':
#                 subscript = True
#             for s in pg.get('subs', list()):
#                 if s.get('is_free_license', None):
#                     free = True
#                 elif s.get('price_in_cents_with_discount', 0) > 0:
#                     purchase = True
#         yield (free, purchase, subscript)
# pprint(Counter(list(scan_values())))
           
pprint(Counter([k for fv in field_values for k in fv.keys()]))

# pprint(Counter([i.get('description', 'MISSING').lower() for fv in field_values for i in fv]))

{'email': '', 'url': 'http://steamcommunity.com/app/10'}

{'email': 'contact@exosyphenstudios.com', 'url': 'http://www.exosyphen.com/page_contact.html'}

{'email': 'poweroflove1801@163.com', 'url': ''}

{'email': 'help@acidwizardstudio.com', 'url': ''}

{'email': 'support@childrenofmorta.com', 'url': 'http://www.childrenofmorta.com/'}

{'email': 'contact@onion-core.com', 'url': 'http://onion-core.com/contact-us/'}

{'email': 'support@spicyhorsegames.zendesk.com', 'url': 'http://support.spicyhorse.com'}

{'email': 'epicdreams90@gmail.com', 'url': 'http://nobodyshot.ru/'}

{'email': 'support@slitherine.co.uk', 'url': 'www.slitherine.com'}

{'email': 'custservice@codemasters.com', 'url': ''}

{'email': 'Dan@BackToBasicsGaming.com', 'url': 'http://www.backtobasicsgaming.com/#!contact/c6th'}

{'email': 'support@gametology.net', 'url': 'www.gametology.net'}

Counter({'email': 12037, 'url': 12037})


## Categories

Each of these should be boolean column (from list of dict's, check 'description')

* Co-op:Co-op|Local Co-op|Online Co-op
* Multiplayer:Cross-Platform Multiplayer|Local Multi-Player|Multi-player|Online Multi-Player|Shared/Split Screen
* In-App Purchases
* Includes Source SDK
* Includes level editor
* MMO
* Single-player
* VR Support

## Genres

Like categories, it's a list of dicts. Use the dict['description'] for text and discretize:

If description is empty string, then ignore it.

All descriptions not described below are GenreIsOther

These descriptions become GenreIsXXX:

* indie
* action
* adventure
* casual
* strategy
* rpg
* simulation
* early access
* free to play
* sports
* racing
* massively multiplayer

These description become GenreIsNotGame:

* utilities
* design & illustration
* animation & modeling
* software training
* education
* audio production
* video production
* web publishing
* photo editing
* accounting


## Requirements

Applied to linux_requirements, mac_requirements, and pc_requirements

Value is dictionary with possible keys:

* minimum
* recommended

Turn into boolean, i.e.

* LinuxReqsHaveMin = True if val.get('minimum', '') else False
* LinuxReqsHaveRec = True if val.get('recommended', '') else False
* MacReqsHaveMin = True if val.get('minimum', '') else False
* MacReqsHaveRec = True if val.get('recommended', '') else False
* PCReqsHaveMin = True if val.get('minimum', '') else False
* PCReqsHaveRec = True if val.get('recommended', '') else False


## Package Groups

3 boolean columns: FreeVerAvail, PurchaseAvail, SubscriptionAvail

Code for all:

````
def package_groups(rec):
    pgs = rec.get("data", dict()).get("package_groups", list)
    free, purchase, subscript = False, False, False
    for pg in pgs:
        if pg.get('is_recurring_subscription', '') == 'true':
            subscript = True
        for s in pg.get('subs', list()):
            if s.get('is_free_license', None):
                free = True
            elif s.get('price_in_cents_with_discount', 0) > 0:
                purchase = True
    return (free, purchase, subscript)
````
